In [ ]:
%matplotlib widget 

In [ ]:
import numpy as np
from numpy import loadtxt
import pandas as pd
import time
import matplotlib
import matplotlib.pyplot as plt
import pyvisa as visa
from matplotlib import animation
from time import sleep
from matplotlib.animation import FuncAnimation
#Importing the functions from the module and the pulses dictionaries
from Sweeping import *
from Dictionaries import *
from PulseFiles import *
from Instrument import *
import pyqtgraph as pg 
import nidaqmx
import pickle

# Instrument settings and CSV file path Dictionary

In [ ]:
AWG_Settings= {
    'Visa Resource Name': 'TCPIP0::ibn3-036.ibn-net.kfa-juelich.de::hislip0::INSTR',
    'Voltage Amplitude':700,
    'Clock Sample Frecuency':6000000000,
    'Output_Channel': 1,
    'Mode': 'STS',
    'Trigger In Threshold' : 1,
    'Output Rout': 'DAC', #DAC output rout only has the BNC outputs: Direct Out and (Averaged) Direct Out
    'Data Directory': r'D:\Alejandro\Pulses\diode measurements\Pulse Scheme CSV\Measurements\1nano'
}

In [ ]:
M8190A = VisaR(AWG_Settings,60000)

In [ ]:
M8190A.query('SYST:ERR?')

In [ ]:
Initialization(M8190A,AWG_Settings)

In [ ]:
M8190A.write('SEQ:DEL:ALL')

# Pulses Dictionaries

In [ ]:
Pump = {
    'Name':'Pump',
    'Amplitude':40,
    'Start time':2,
    'End time':2e-10,
    'Start Duration':1,
    'End Duration':60,
    'Sweep time':0,
    'Sweep Duration':0
}

In [ ]:
ProbeA = {
    'Name':'ProbeA',
    'Amplitude':25,
    'Start time':0,
    'End time':4,
    'Start Duration':1,
    'End Duration':60,
    'Sweep time':1,
    'Sweep Duration':0
}

In [ ]:
ProbeB = {
    'Name':'ProbeB',
    'Amplitude':0,
    'Start time':0,
    'End time':12,
    'Start Duration':1,
    'End Duration':6,
    'Sweep time':0,
    'Sweep Duration':0
}

# Pulse Schemes Dictionaries and measurement path file

In [ ]:
PulseScheme_A =  {
    'Name' : 'PulseScheme_A',
    'Pulse Scheme': [Pump,ProbeA],
    'Number of repetitions': 1, #10000
    'Measurement file Path': r'D:\Alejandro\Pulses\diode measurements\TList\1nano'
}

In [ ]:
PulseScheme_B  = {
    'Name' : 'PulseScheme_B',
    'Pulse Scheme': [ProbeB,ProbeB],
    'Number of repetitions': 1,
    'Measurement file Path': r'D:\Alejandro\Pulses\diode measurements\TList\1nano'
}

In [ ]:
location = PulseScheme_A['Measurement file Path']

# Number of Samples given the AWG Sampling rate and the Reference Signal Frequency
## Sr = 6Ghz for 5 ms

In [ ]:
sam = int(6e9*5e-3)

#sam /480

#M8190A.write('FREQ:RAST 6000000000')

In [ ]:
sam

In [ ]:
pul2 = Sweep(PulseScheme_B['Pulse Scheme'],10,0,5,sam)

In [ ]:
%matplotlib inline

In [ ]:
%matplotlib widget 

In [ ]:
location = PulseScheme_A['Measurement file Path']

In [ ]:
sam = int(6e9*5e-3)
start = 0
middle = 5
end = 10
#pul0 = Sweep_teil(PulseScheme_A,10,start,5,sam)
#pulm = Sweep_teil(PulseScheme_A,10,middle,5,sam)
#pulend = Sweep_teil(PulseScheme_A,10,end,5,sam)


#Plot
fig,ax= plt.subplots()
ax.grid(True)

plt.plot(pul0[1],pul0[0],label= 'Step {b}'.format(b = start),color='green')
plt.plot(pulm[1],pulm[0],label= 'Step {b}'.format(b = middle),color='black')
plt.plot(pulend[1],pulend[0],label= 'Step {b}'.format(b = end),color='blue')


ax.set_ylabel('V  [mV]')
ax.set_xlabel(r't [ms]')
ax.set_title('Cycle A at step at 3 different sweeping steps') #Voltage Offset = 3mV, Marker Ampl = 50mV 
ax.legend(loc=0)
#plt.legend(bbox_to_anchor=(2.05, 1), loc='upper left', borderaxespad=0.)
plt.show()
fig.savefig('Cycle A,3 different steps.jpeg',bbox_inches='tight')

In [ ]:
sam = int(6e9*5e-3)
step = 30
pul1 = Sweep_teil(PulseScheme_B,30,step,5,sam)


#Plot
fig,ax= plt.subplots()
ax.grid(True)

plt.plot(pul1[1],pul1[0],label= '{b} samples'.format(b = sam),color='red')

ax.set_ylabel('V  [mV]')
ax.set_xlabel(r't [ms]')
ax.set_title('Cycle B ') #Voltage Offset = 3mV, Marker Ampl = 50mV 
#ax.legend(loc=0)
#plt.legend(bbox_to_anchor=(2.05, 1), loc='upper left', borderaxespad=0.)
plt.show()
fig.savefig('Cycle B.jpeg')

In [ ]:
step = 30

pul1 = Sweep_teil(PulseScheme_A,30,step,5000000,sam)

fig,ax= plt.subplots()
ax.grid(True)

#plt.plot(timeMs,dataMs[1]*1000,label= 'Marker Channel {chan}'.format(chan=DAQ_Settings1['Analog Channel Input Marker']),color='purple')
plt.plot(pul1[1],pul1[0],label= '{b} samples'.format(b = sam),color='orange')

ax.set_ylabel('V  [mV]')
ax.set_xlabel('t  [ns]')
ax.set_title('Cycle A at step {a}, {rep} repetitions'.format(a = step,rep = PulseScheme_A['Number of repetitions'])) #Voltage Offset = 3mV, Marker Ampl = 50mV  
#ax.legend(loc=0)
#plt.legend(bbox_to_anchor=(2.05, 1), loc='upper left', borderaxespad=0.)
plt.show()
#fig.savefig(r'{loc}\Cycle A {reps} reps.jpeg'.format(loc =location, reps = PulseScheme_A['Number of repetitions']),bbox_inches='tight')

In [ ]:
step = 30

#pul1 = Sweep_teil(PulseScheme_AutocorrelationA,30,step,5000,sam)


fig,ax= plt.subplots(2)


ax[0].grid(True)
ax[1].grid(True)
fig.suptitle('Pulses Loaded into the AWG')


ax[0].plot(pul1[1],pul1[0],label= 'Pulse A',color='blue')
ax[1].plot(pul2[1],pul2[0],label= 'Pulse B',color='red')
ax[0].set_ylabel('V  [mV]')
ax[0].set_xlabel(r't  [$\mu$s]')
ax[1].set_ylabel('V  [mV]')
ax[1].set_xlabel(r't  [ms]')

#ax.set_title('Pulses Loaded into the AWG')

fig.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)
plt.show()

#fig.savefig('Pulse_scheme_15ms_7500000',optimize=True,bbox_inches='tight')

# Dummy File Loading

In [ ]:
Segment_File(M8190A,'D:\\Alejandro\\Pulses\\Dict\\S5\\SegmentB_30000000_0.csv',1)
M8190A.write('INNIT:IMM')
time.sleep(5)
M8190A.write('ABOR')

In [ ]:
M8190A.write('TRAC:DEL:ALL')
M8190A.write('SEQ:DEL:ALL')

In [ ]:
M8190A.write('SEQ:DEL:ALL')

In [ ]:
sweeping_run_a = Sweeping_Single_List_File_teil(PulseScheme_A,30,5000000,sam,0,30,AWG_Settings,'A')[0]
#sweeping_runb = Sweeping_Single_List_File_teil(PulseScheme_B,30,5000,sam,0,30,AWG_Settings8,'B')[0]

In [ ]:
#exporting pickle 

pickle.dump(sweeping_run_a, open(r"{dict}\cycleA_31steps_5ms_1nano_probe.p".format(dict = PulseScheme_A['Measurement file Path']), "wb"))  # save it into a file named sweepingA.p
#pickle.dump(sweeping_runb, open("sweepingB_30steps_5ms_10micro_probe.p", "wb"))  # save it into a file named sweepingB.p


In [ ]:
#importing pickle

#sweeping_runa = pickle.load( open(r"{dict}\cycleA_31steps_5ms_1nano_probe.p".format(dict = PulseScheme_A['Measurement file Path']), "rb"))
sweeping_runb = pickle.load(open("sweepingB_30steps_5ms_10micro_probe.p", "rb"))

In [ ]:
sweeping_run_a

In [ ]:
Lock_In_Settings = {
'Time Constant' : '100ms',
'Sensitivity' : '500 microVolts'
'Noise' : 'Normal'
'db' : 18
'Modulation': '50Hz'
}

In [ ]:
DAQ_time = 10
tc = '30ms'
sns = '100 microV'

Diode_Signal = Sequence_Loader_File_DAQ_np(M8190A,DAQ_Settings1,1000,DAQ_time,sweeping_run_a,sweeping_runb)

In [ ]:
fig,ax= plt.subplots()
ax.grid(True)
for i in range(0,len(Diode_Signal)):
    plt.plot(Diode_Signal[i][1],Diode_Signal[i][0],label= 'Step {st}'.format(st = i ))

ax.set_ylabel(r'V  [V]')
ax.set_xlabel('t  [s]')
ax.set_title(r' Diode Signal Measured by DAQ,  {b} samples, 1ns probe duration'.format(b = len(Diode_Signal[0][1]) ))
#ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)
#ax.legend(loc=0)
plt.show()
fig.savefig(r'{loc}\Diode Signal time cstant = {tcnt}, sensitivity 500microV,{f}DAQ time, 5ms scheme, probe duration 1nano,31 steps,80k .jpeg'.format(loc = location, tcnt =tc, f=DAQ_time),bbox_inches='tight')

In [ ]:
location

In [ ]:
for i in range(0,len(Diode_Signal)):
    np.savetxt(r'{loc}\diode_signal_step{stp}_{f}sdaqtime_5ms_100mstc_100microV.csv'.format(loc = location ,stp = i,f = DAQ_time), Diode_Signal[i][0], delimiter=',') 

chopped_measurement_data = np.zeros((len(Diode_Signal)),  dtype=object)
for i in range(0,len(Diode_Signal)):
    chopped_measurement_data[i] = Diode_Signal[i][0][2500:] 

avr = np.array([np.average(i) for i in chopped_measurement_data])
np.savetxt(r'{loc}\averaged signal_30steps_{f}sdaqtime_100mstc_100microV.csv'.format(loc =location,f = DAQ_time),avr,delimiter=',')

In [ ]:
np.savetxt(r'averaged signal_30steps_1nano_10sdaqtime_2000rep.csv',avr,delimiter=',')

In [ ]:
time_delay = np.linspace(-2,2,31)

In [ ]:
#change this to V regime
fig,axes = plt.subplots()

axes.grid(True)

plt.plot(time_delay,avr,label=r'1ns Probe Width ',color='brown')

axes.set_xlabel(r'time delay [ns]')
axes.set_ylabel(r'Diode Signal [V]')
axes.set_title('Auto-Correlation 31 steps ')

axes.legend(loc = 0)
plt.show()

fig.savefig(r'{loc}\Autocorrelation function 31 steps reps 1nano probe2s, tc = 100ms , sns = 100microV,{f}daqs.jpeg'.format(loc = location,f = DAQ_time),bbox_inches='tight')

# Load numpy array from csv file


In [ ]:
# load array
avrg_1point5 = loadtxt('averaged signal_30steps_onepoint5.csv',delimiter= ",")
avrg_point1 = loadtxt('averaged signal_30steps_point1.csv',delimiter = ',')
avrg_1 = loadtxt('averaged signal_30steps_35seconds.csv', delimiter=',')
avrg_pointfive = loadtxt('averaged signal_30steps_35seconds_pointfive.csv', delimiter=',')
avrg_10micro = loadtxt('averaged signal_30steps_10micro.csv', delimiter= ',')
# print the array


In [ ]:
fig,axes = plt.subplots()

axes.grid(True)
#plt.plot(time_delay,avrg_1,label='Probe width 1',color='black')
plt.plot(time_delay,avrg_10micro,label=r'10 $\mu$s Probe Width ',color='green')
plt.plot(time_delay,avrg_1,label='Probe width 1 ms',color='blue')
#plt.plot(time_delay,avrg_pointfive,label = 'Probe width 0.5', color = 'purple')
#plt.plot(time_delay,avrg_point1,label='Probe width 0.1',color='red')


axes.set_xlabel('time delay [ms]')
axes.set_ylabel(r'Diode Signal [V]')
axes.set_title('Auto-Correlation Signal, 30 Steps ')

axes.legend(loc = 0)
plt.show()

#fig.savefig('Autocorrelation 1ms Width scheme.jpeg',bbox_inches='tight')

In [ ]:
fig,axes = plt.subplots()

axes.grid(True)
plt.plot(time_delay,avr,label='Probe width 2',color='black')
plt.plot(time_delay,avrg_1point5,label='Probe width 1.5',color='green')
plt.plot(time_delay,avrg_1,label='Probe width 1',color='blue')
plt.plot(time_delay,avrg_pointfive,label = 'Probe width 0.5', color = 'purple')
plt.plot(time_delay,avrg_point1,label='Probe width 0.1',color='red')


axes.set_xlabel('time delay [ms]')
axes.set_ylabel(r'Diode Signal [V]')
axes.set_title('Auto-Correlation Signal, 30 Steps ')

axes.legend(loc = 0)
plt.show()

#fig.savefig('Autocorrelation 2 width.jpeg',bbox_inches='tight')

In [ ]:
do 1 more 1.5 probe plot

In [ ]:
 np.where(haber[0][1] > 3 )

In [ ]:
%matplotlib inline

In [ ]:
import glob
print(glob.glob('D:\Alejandro\Pulses\diode measurements\Pulse Scheme CSV\AutoCorrelation, ms regime, 0,1 probe duration\*csv'))
#print(glob.glob("/home/adam/*"))

In [ ]:
a = glob.glob('D:\Alejandro\Pulses\diode measurements\Pulse Scheme CSV\AutoCorrelation, ms regime, 0,1 probe duration\*csv')

In [ ]:
a

# Tiling

In [ ]:
Pump = {
    'Name':'Pump',
    'Amplitude':40,
    'Start time':2e-10,
    'End time':2e-10,
    'Start Duration':2,
    'End Duration':6,
    'Sweep time':0,
    'Sweep Duration':0
}

In [ ]:
ProbeAutoA = {
    'Name':'ProbeAutoA',
    'Amplitude':25,
    'Start time':0,
    'End time':4,
    'Start Duration':1,
    'End Duration':6,
    'Sweep time':1,
    'Sweep Duration':0
}

In [ ]:
ProbeAutoB = {
    'Name':'ProbeAutoB',
    'Amplitude':0,
    'Start time':0,
    'End time':12,
    'Start Duration':1,
    'End Duration':6,
    'Sweep time':0,
    'Sweep Duration':0
}

In [ ]:
ProbeAutoC = {
    'Name':'ProbeAutoB',
    'Amplitude':35,
    'Start time':3,
    'End time':12,
    'Start Duration':3,
    'End Duration':6,
    'Sweep time':0,
    'Sweep Duration':0
}

In [ ]:
AutoCorSchemeA = [Pump, ProbeAutoA]
AutoCorSchemeB = [ProbeAutoB, ProbeAutoB]
AutoCorSchemeC = [Pump,ProbeAutoC]
#step = 0

In [ ]:
ProbeAutoC = {
    'Name':'ProbeAutoB',
    'Number of Repetitions': 1,
    'Amplitude':35,
    'Start time':6,
    'End time':12,
    'Start Duration':3,
    'End Duration':6,
    'Sweep time':0,
    'Sweep Duration':0
}

In [ ]:
PulseSchemetest =  {
    'Name' : 'PulseSchemetest',
    'Pulse Scheme': [Pump,ProbeAutoC],
    'Number of repetitions': 4,
}

In [ ]:
def Sweepteil(PulList,P,p,t,N):
    
    """Function that perfoms or not a Sweep, depending on the Sweep dictionary keys, for the Pulse Scheme PulList.
    
    This function is a generalization of the sweep0(Pulse,P,p,t).
    It calls it and maps it over every Pulse in the list of pulses PulList.
    If two pulses overlap, their amplitude is added up.
    P gives the number of sample points for the overal pulse scheme, not to be confused with dt, the sweeping time step given by P
    The pulse sequence is generated in the order that the pulses have within the PulseList.
    """
    
    #time interval
    time = np.linspace(0,t,N)
    
    #Each pulse will be an array, and an entry of a bigger array (we have an array of arrays)
    pularray = np.array([sweep0(Pulse,P,p,t,N) for Pulse in PulList['Pulse Scheme']])
        
    #the final pulse scheme will be the overlap of each individual pulse, given by the sum of their arrays    
    pulscheme0 = np.sum(pularray,0)

    #we divide the pulse scheme in order to play it several times
    teil = int(pulscheme0.size/PulList['Number of repetitions'])
    pulscheme = np.tile(pulscheme0[:teil],PulList['Number of repetitions'])
    
        
    return pulscheme, time

In [ ]:
def Sweept(PulList,P,p,t,N,rep):
    
    """Function that perfoms or not a Sweep, depending on the Sweep dictionary keys, for the Pulse Scheme PulList.
    
    This function is a generalization of the sweep0(Pulse,P,p,t).
    It calls it and maps it over every Pulse in the list of pulses PulList.
    If two pulses overlap, their amplitude is added up.
    P gives the number of sample points for the overal pulse scheme, not to be confused with dt, the sweeping time step given by P
    The pulse sequence is generated in the order that the pulses have within the PulseList.
    """
    
    #time interval
    time = np.linspace(0,t,N)
    
    #Each pulse will be an array, and an entry of a bigger array (we have an array of arrays)
    pularray = np.array([sweep0(Pulse,P,p,t,N) for Pulse in PulList['Pulse Scheme']])
        
    #the final pulse scheme will be the overlap of each individual pulse, given by the sum of their arrays    
    pulscheme0 = np.sum(pularray,0)

    #we divide the pulse scheme in order to play it several times
    teil = int(pulscheme0.size/rep)
    pulscheme = np.tile(pulscheme0[:teil],rep)
    
        
    return pulscheme, time

In [ ]:
pul = Sweept(PulseSchemetest,10,0,100,7500000,10)

In [ ]:
n = 10
pul = Sweept(PulseSchemetest,10,0,100,7500000,n)
#teil = int(pul[0].size/n)
#huh = np.tile(pul[0][:teil],n)

fig,axes = plt.subplots()
axes.grid(True)

plt.plot(pul[1],pul[0],label = 'Step 0' )
#plt.plot(tim2,pul1,label = 'part2' )

axes.set_xlabel('t [ms]')
axes.set_ylabel('v[mV]')
axes.set_title('Auto-Correlation Cycle A repeated {a} times'.format( a = n ))

#axes.legend(loc = 0)
plt.show()

#fig.savefig('{a} pulses per segment.png'.format( a = n))

In [ ]:
fig,axes = plt.subplots()

axes.grid(True)

plt.plot(pul[1],pul[0],label = '{a} samples'.format(a = len(pul[0])))
#plt.plot(tm,pul,label = '{a} samples'.format(a = len(pul)))

axes.set_xlabel('t [ms]')
axes.set_ylabel('v[mV]')
axes.set_title('Auto-Correlation Pulse Scheme at step 0')

axes.legend(loc = 0)
plt.show()

In [ ]:
dataM, timeM = DAQ_Measuring(DAQ_Settings1,1000,30,M8190A)

fig,ax= plt.subplots()
ax.grid(True)

#plt.plot(timeMs,dataMs[1]*1000,label= 'Marker Channel {chan}'.format(chan=DAQ_Settings1['Analog Channel Input Marker']),color='purple')
plt.plot(timeM,dataM*1000,label= '{b} samples'.format(b=len(dataM)),color='orange')

ax.set_ylabel('V  [mV]')
ax.set_xlabel('t  [ms]')
ax.set_title('AutoCorrelation Scheme Measured with DAC, Overlaping Step ') #Voltage Offset = 3mV, Marker Ampl = 50mV 
ax.legend(loc=0)
#plt.legend(bbox_to_anchor=(2.05, 1), loc='upper left', borderaxespad=0.)
plt.show()
#fig.savefig('AutoCorrelation Scheme Meaasured with DAC, Overlaping Step.pdf',bbox_inches='tight')

In [ ]:
A= 5
B = 7

f1 = 10 * 2*np.pi
f2 = 20* 2*np.pi
phase = 0

In [ ]:
def ip(t):
    out = A * np.cos( f1 * t + phase )
    return out

In [ ]:
def ref(t):
    out = B * np.cos( f2 * t + phase )
    return out

In [ ]:
t = np.linspace(0,100,200)

In [ ]:
Vo = np.array([ip(x) for x in t ])
Vr = np.array([ref(x) for x in t ])



In [ ]:


#Plot
fig,ax= plt.subplots()
ax.grid(True)

plt.plot(t,Vo,label= 'Input Signal',color='orange')
#plt.plot(t,Vo,label= 'Reference Signal',color='blue')

#ax.set_ylabel('V  [mV]')
ax.set_xlabel(r't [s]')
ax.set_title('Input Signal') #ax.legend(loc=0)
#plt.legend(bbox_to_anchor=(2.05, 1), loc='upper left', borderaxespad=0.)
plt.show()
fig.savefig('Input Signal.jpeg')

In [ ]:


#Plot
fig,ax= plt.subplots()
ax.grid(True)

#plt.plot(t,Vo,label= 'Input Signal',color='orange')
plt.plot(t,Vr,label= 'Reference Signal',color='blue')

#ax.set_ylabel('V  [mV]')
ax.set_xlabel(r't [s]')
ax.set_title('Reference Signal') 
#ax.legend(loc=0)
#plt.legend(bbox_to_anchor=(2.05, 1), loc='upper left', borderaxespad=0.)
plt.show()
fig.savefig('Reference Signal.jpeg')

In [ ]:
modul = Vo * Vr

fig,ax= plt.subplots()
ax.grid(True)

#plt.plot(t,Vo,label= 'Input Signal',color='orange')
plt.plot(t,modul,label= 'Product of Signals',color='black')

#ax.set_ylabel('V  [mV]')
ax.set_xlabel(r't [s]')
ax.set_title('Product Signal') 
#ax.legend(loc=0)
#plt.legend(bbox_to_anchor=(2.05, 1), loc='upper left', borderaxespad=0.)
plt.show()
fig.savefig('Product Signal different frequencies.jpeg')

In [ ]:
modul = Vo * Vr

fig,ax= plt.subplots()
ax.grid(True)

#plt.plot(t,Vo,label= 'Input Signal',color='orange')
plt.plot(t,modul,label= 'Product of Signals',color='black')

#ax.set_ylabel('V  [mV]')
ax.set_xlabel(r't [s]')
ax.set_title('Product Signal') 
#ax.legend(loc=0)
#plt.legend(bbox_to_anchor=(2.05, 1), loc='upper left', borderaxespad=0.)
plt.show()
fig.savefig('Product Signal same frequencies.jpeg')

Self Updating Plotting

In [ ]:
import time
import random
import pyqtgraph as pg
from collections import deque
from pyqtgraph.Qt import QtGui, QtCore
import numpy as np

In [ ]:
test1= np.random.randint(0,11,50)
avrg = np.average(test1)

In [ ]:
test1

In [ ]:
avrg

In [ ]:
def Sim(iter):
    for i in range(0,iter):
        test = np.random.randint(0,11,50)
        average = np.average(test)
        print(average)
        time.sleep(5)

In [ ]:
Sim(5)

In [ ]:
class Graph:
    def __init__(self, ):
        self.dat = deque()
        self.maxLen = 50#max number of data points to show on graph
        self.app = QtGui.QApplication([])
        self.win = pg.GraphicsWindow()
       
        self.p1 = self.win.addPlot(colspan=2)
        self.win.nextRow()
        self.p2 = self.win.addPlot(colspan=2)
        self.win.nextRow()
        self.p3 = self.win.addPlot(colspan=2)
       
        self.curve1 = self.p1.plot()
        self.curve2 = self.p2.plot()
        self.curve3 = self.p3.plot()
       
        graphUpdateSpeedMs = 100 #greater slower
        timer = QtCore.QTimer()#to create a thread that calls a function at intervals
        timer.timeout.connect(self.update)#the update function keeps getting called at intervals
        timer.start(graphUpdateSpeedMs)   
        QtGui.QApplication.instance().exec_()
       
    def update(self):
        if len(self.dat) > self.maxLen:
            self.dat.popleft() #remove oldest
        self.dat.append(random.randint(0,100)); 

        self.curve1.setData(self.dat)
        self.curve2.setData(self.dat)
        self.curve3.setData(self.dat)
        self.app.processEvents()  
       

if __name__ == '__main__':
    g = Graph()

In [ ]:
1+1

In [ ]:
# Import libraries
from numpy import *
from pyqtgraph.Qt import QtGui, QtCore
import pyqtgraph as pg
import serial

# Create object serial port
portName = "COM12"                      # replace this port name by yours!
baudrate = 9600
ser = serial.Serial(portName,baudrate)

### START QtApp #####
app = QtGui.QApplication([])            # you MUST do this once (initialize things)
####################

win = pg.GraphicsWindow(title="Signal from serial port") # creates a window
p = win.addPlot(title="Realtime plot")  # creates empty space for the plot in the window
curve = p.plot()                        # create an empty "plot" (a curve to plot)

windowWidth = 500                       # width of the window displaying the curve
Xm = linspace(0,0,windowWidth)          # create array that will contain the relevant time series     
ptr = -windowWidth                      # set first x position

# Realtime data plot. Each time this function is called, the data display is updated
def update():
    global curve, ptr, Xm    
    Xm[:-1] = Xm[1:]                      # shift data in the temporal mean 1 sample left
    value = ser.readline()                # read line (single value) from the serial port
    Xm[-1] = float(value)                 # vector containing the instantaneous values      
    ptr += 1                              # update x position for displaying the curve
    curve.setData(Xm)                     # set the curve with this data
    curve.setPos(ptr,0)                   # set x position in the graph to 0
    QtGui.QApplication.processEvents()    # you MUST process the plot now

### MAIN PROGRAM #####    
# this is a brutal infinite loop calling your realtime data plot
while True: update()

### END QtApp ####
pg.QtGui.QApplication.exec_() # you MUST put this at the end
##################


In [ ]:
from pyqtgraph.Qt import QtGui, QtCore
import numpy as np
import pyqtgraph as pg

# Set graphical window, its title and size
win = pg.GraphicsWindow(title="Sample process")
win.resize(1000,600)
win.setWindowTitle('pyqtgraph example')

# Enable antialiasing for prettier plots
pg.setConfigOptions(antialias=True)

# Random data process
p6 = win.addPlot(title="Updating plot")
curve = p6.plot(pen='y')
#data = np.random.normal(size=(10,1000)) #  If the Gaussian distribution shape is, (m, n, k), then m * n * k samples are drawn.
test = np.random.randint(0,11,50)
data = np.average(test)


# plot counter
ptr = 0 

# Function for updating data display
def update():
    global curve, data, ptr, p6
    curve.setData(data[ptr%10])
    if ptr == 0:
        p6.enableAutoRange('xy', False)  ## stop auto-scaling after the first data set is plotted
    ptr += 1

# Update data display    
timer = QtCore.QTimer()
timer.timeout.connect(update)
timer.start(50)


## Start Qt event loop unless running in interactive mode or using pyside.
if __name__ == '__main__':
    import sys
    if (sys.flags.interactive != 1) or not hasattr(QtCore, 'PYQT_VERSION'):
        QtGui.QApplication.instance().exec_()

In [ ]:
        test = np.random.randint(0,11,50)
        average = np.average(test)

In [21]:
 np.random.normal(size=(10,1000))

array([[-1.5071184 ,  0.05615916,  0.71447734, ..., -0.70843724,
         2.0222635 ,  1.75433422],
       [ 1.60916638,  2.45087457,  1.09025169, ..., -1.96940005,
        -1.15328075,  2.30088499],
       [-0.23473813,  1.10185945,  2.04544019, ...,  2.54939189,
         0.68097971,  0.01745949],
       ...,
       [-1.32431336,  0.41571231,  0.11251223, ..., -0.32220321,
         0.12202609,  1.07717266],
       [-1.52682644, -0.02048585,  0.59484088, ..., -0.38080654,
         0.28759156, -0.77912959],
       [-0.43640508,  0.62697164, -1.4682728 , ...,  1.01614026,
         0.3660703 ,  0.58256746]])